# Analyse der Verkehrsunfälle im Kreis 11

In [24]:
!pip install --upgrade pip

Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/a4/6d/6463d49a933f547439d6b5b98b46af8742cc03ae83543e4d7688c2420f8b/pip-21.3.1-py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [25]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot')

!pip install folium
import folium # plotting library
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime


# 1 Datensatz einlesen

In [26]:
df_11 = pd.read_csv('./data/verkehrsunfaelle_2020_kreis11.csv')
df_11.head()

,index,AccidentUID,Quartier,AccidentType,AccidentType_de,AccidentSeverityCategory,AccidentSeverityCategory_de,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_de,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,CantonCode,MunicipalityCode,AccidentYear,AccidentMonth,AccidentMonth_de,AccidentWeekDay,AccidentWeekDay_de,AccidentHour,AccidentHour_text,AccidentLocation_WGS84_E,AccidentLocation_WGS84_N,Kreis,Jahreszeit,ZeitKategorie
0,14,9BFFA15CC2414409E05329B9D80A8565,Affoltern,at4,Einbiegeunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2680556,1252485,ZH,261,2020,1,Januar,aw405,Freitag,13.0,13h-14h,47.418246,8.506139,Kreis 11,Winter,Nachmittag
1,22,9C02A2D8710302EDE05329B9D80AC5D9,Oerlikon,at0,Schleuder- oder Selbstunfall,as4,Unfall mit Sachschaden,False,False,False,rt432,Hauptstrasse,2683803,1250936,ZH,261,2020,1,Januar,aw406,Samstag,13.0,13h-14h,47.403910,8.548872,Kreis 11,Winter,Nachmittag
2,38,9DA9A08B38DAAD65E05329B9D80A97B1,Oerlikon,at7,Parkierunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2683541,1251753,ZH,261,2020,1,Januar,aw401,Montag,7.0,07h-08h,47.411291,8.545554,Kreis 11,Winter,Rushhour_Morgen
3,67,A4CEB8229AD1E67EE05328B9D80A8A77,Oerlikon,at3,Abbiegeunfall,as4,Unfall mit Sachschaden,False,False,False,rt432,Hauptstrasse,2683881,1251129,ZH,261,2020,1,Januar,aw403,Mittwoch,19.0,19h-20h,47.405636,8.549941,Kreis 11,Winter,Rushhour_Abend
4,69,9BAF6D9E8FBAF4F0E05329B9D80AD7E9,Oerlikon,at0,Schleuder- oder Selbstunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2684282,1251091,ZH,261,2020,1,Januar,aw403,Mittwoch,20.0,20h-21h,47.405243,8.555246,Kreis 11,Winter,Abend


# 2 Konfigurationsdaten

GeoJSON Datei für die Chloropleth Plots

In [27]:
map_kreis11_path = './data/Stadtkreise_ZH/stzh.adm_stadtkreise_kreis11.geojson'

In [28]:
# Zürich Kreis 11 longitude und latitude
# entspricht in etwa dem Zentrum des Kreises und wird beim Erstellen einer Karte mitgegeben um diese zu zentrieren
Kreis11_lat = 47.418792488745325
Kreis11_lon = 8.530429483637539

# 3 Übersichts Chloropleth Plot

Der Kreis 11 besteht aus den Quartieren Oerlikon, Seebach und Affoltern. Auf dieser Karte soll dargestellt werden wie hoch die Unfalldichte in den einzelnen Quartieren ist.

In [29]:
x = df_11[['Quartier', 'AccidentUID']].groupby(['Quartier'])
z = x.count().reset_index()

# da die höchste Farbstufe in den Chloroplezh Paletten immer sehr dunkel ist: diese lieber vermeiden
z.loc[len(z.index)]=['',1.05*z.AccidentUID.max()]   
z

,Quartier,AccidentUID
0,Affoltern,278.00
1,Oerlikon,279.00
2,Seebach,243.00
3,,292.95


In [30]:
map_11 = folium.Map(location=[Kreis11_lat, Kreis11_lon], zoom_start=13)
folium.Choropleth(
    geo_data= map_kreis11_path,
    name='Number of Accidents',
    data=z,
    columns=['Quartier','AccidentUID'],
    key_on='feature.properties.qname',
    fill_color='Greys',
    fill_opacity=0.7,
    legend_name='Number of Accidents'
).add_to(map_11) 

folium.LayerControl().add_to(map_11)

map_11

# 4 Cluster Plots mit Anzeige des Schweregrads und der Beteiligten


Nun soll eine Karte erzeugt werden, in der alle Unfälle je nach Zoom Level entweder einzeln als Marker oder gruppiert in Clustern angezeigt werden. Wird auf ein Cluster draufgeklickt, so geht dieser auf und es wird der nächsthöhere Detaillierungsgrad angezeigt.

Die Marker für die einzelnen Unfälle zeigen folgende Informationen an:
* Position: Ort des Unfalls
* Farbe des Markers: Schweregrad des Unfalls (hellblau: nur Sachschaden, hellrot:Leichtverletzte, dunkelrot: Schwerverletzte, schwarz: Tote)
* Icon des Markers: Falls Fussgänger, Motorrad- oder Velofahrer am Unfall beteiligt waren, so wird dies mit einem entsprechenden Icon angezeigt.
* Tooltip: Wenn sich die Maus über einem Marker befindet werden zusätzliche Informationen zum betreffenden Unfall angezeigt (Art des Unfalls, Wochentag, Monat und Tageszeitkategorie)

Im Hintergrund sind zudem immer noch die Angaben zur Unfalldichte pro Quartier aus dem obigen Chloropleth Plot ersichtlich.

In [31]:
# Um die Farbcodierung des Markers richtig hinzubekommen werden wird überprüft welche Kategorie welchem Schweregrad entspricht
# (allerdings haben nicht alle Kreise Unfaelle der Kategorie as1, die also noch zusätzlich hinzunehmen)
df_11.groupby(['AccidentSeverityCategory', 'AccidentSeverityCategory_de']).size().to_frame()

,,0
AccidentSeverityCategory,AccidentSeverityCategory_de,
as2,Unfall mit Schwerverletzten,21
as3,Unfall mit Leichtverletzten,148
as4,Unfall mit Sachschaden,631


In [32]:
#help(folium.Icon)

Um nicht mehrere Male dasselbe programmieren zu müssen soll eine Funktion geschrieben werden, die eine beliebige Teilmenge der Unfalldaten entgegennimmt und die ganze Verarbeitung dieser Daten durchführt.

Die folgende Funktion nimmt also einen Datensatz von Unfalldaten für den Kreis 11, z.B. alle Velounfälle, alle Unfälle im Winter, alle Unfälle während der Rushhour am Abend etc. (oder auch alle Daten) und stellt diese auf einer Karte dar. Die resultierende Karte wird einerseits als Datei abgespeichert und zudem als Returnwert zurückgegeben.

Grundsätzlich können der Funktion auch Daten aus anderen Kreisen übergeben werden, auch diese werden korrekt auf der Karte angezeigt. Einzig die Hintergrundinformation bezüglich der allgemeinen Unfalldichte pro Quartier (Chloropleth Layer) wird unabhängig von den Daten immer nur für den Kreis 11 angezeigt. Dies könnte aber mit wenig Aufwand auch noch integriert werden.


In [33]:
def create_Kreis11_ClusterPlot(df, outputfilename):

    map_11 = folium.Map(location=[Kreis11_lat, Kreis11_lon], zoom_start=13)    
    
    # Kalibrieren der Inputdaten für den Chloropleth Layer aus dem gegebenen Datensatz
    x = df[['Quartier','AccidentUID']].groupby(['Quartier'])
    z = x.count().reset_index()
    
    # da die höchste Farbstufe in den Chloroplezh Paletten immer sehr dunkel ist diese lieber vermeiden:
    z.loc[len(z.index)]=['',1.05*z.AccidentUID.max()]


    # Jeder Marker soll eine Farbe haben, die den Schweregrad des Unfalls angibt
    # hellblau: nur Sachschaden, hellrot: Leichtverletzte, dunkelrot: Schwerverletzte, schwarz: Tote
    colorcodes = {'as1': 'black', 'as2':'darkred', 'as3':'lightred', 'as4':'lightblue'}
    df['MarkerColor']=df['AccidentSeverityCategory'].map(colorcodes)

    # Falls bei einem Unfall Fussgänger, Velo- oder Motorradfahrer involviert waren soll dies mit einem Icon im Marker angezeigt werden
    # Achtung: evt. wenn mehrere Kriterien zutreffen, werden die Werte überschrieben, d.h. Reihenfolge der Statements ist wichtig!
    # da wir insbesondere Velos analysieren wollen müssen diese sicher zuletzt geschrieben werden
    df['MarkerIcon']='none'   # default: kein icon ('off' wäre ebenfalls möglich)
    df.loc[df.AccidentInvolvingMotorcycle==True, 'MarkerIcon']='motorcycle'  
    df.loc[df.AccidentInvolvingPedestrian==True, 'MarkerIcon']='male'  
    df.loc[df.AccidentInvolvingBicycle==True, 'MarkerIcon']='bicycle'  

    mc = MarkerCluster(name='Accident Clusters').add_to(map_11)   

    for row in df.itertuples():
        folium.Marker(
            location=[row.AccidentLocation_WGS84_E, row.AccidentLocation_WGS84_N],
            icon=folium.Icon(color=row.MarkerColor, icon=row.MarkerIcon, prefix='fa'),
            tooltip=f"{row.AccidentType_de} ({row.AccidentWeekDay_de}, {row.AccidentMonth_de}, {row.ZeitKategorie})"
        ).add_to(mc)

    folium.Choropleth(
        geo_data= map_kreis11_path,
        name='Number of Accidents',
        data=z,
        columns=['Quartier','AccidentUID'],
        key_on='feature.properties.qname',
        fill_color='Greys',
        fill_opacity=0.7,
        legend_name='Number of Accidents'
    ).add_to(map_11)    
    
    folium.LayerControl(collapsed=False).add_to(map_11) 

    map_11.save(outputfilename)  
    
    return map_11

### 4.1 Cluster Plot mit allen Unfällen

In [34]:
create_Kreis11_ClusterPlot(df_11.copy(), './results/Kreis11_Clustermap_alleUnfaelle.html')

### 4.2 Cluster Plot nur mit den Velo-Unfällen

In [35]:
create_Kreis11_ClusterPlot(df_11[df_11['AccidentInvolvingBicycle'] == True].copy(), 
                           './results/Kreis11_Clustermap_VeloUnfaelle.html')

Beim Anschauen dieser Karte auf Level der einzelnen Marker fällt auf, dass der Anteil an roten Markern, also an Unfällen mit Verletzten, sehr viel höher ist als auf der Karte mit allen Unfällen. Dieser Eindruck wird durch die nachfolgende Tabelle bestätigt. Es macht also sehr viel Sinn, den Velounfällen eine besondere Aufmerksamkeit zu schenken.

In [36]:
df_11[df_11['AccidentInvolvingBicycle'] == True].groupby(['AccidentSeverityCategory', 'AccidentSeverityCategory_de']).size().to_frame()

,,0
AccidentSeverityCategory,AccidentSeverityCategory_de,
as2,Unfall mit Schwerverletzten,8
as3,Unfall mit Leichtverletzten,44
as4,Unfall mit Sachschaden,12


# 5) HeatMaps

Ebenfalls ein sehr gutes Mittel, sich schnell einen Überblick zu verschaffen über die Anzahl der Unfälle, ist eine HeatMap. Diese bietet zwar nicht so viele Möglichkeiten, zusätzliche Informationen zu den Unfallorten verfügbar zu machen wie die Cluster Maps, dafür sind sie beim Herein- und Herauszoomen wesentlich einfacher zu handhaben, indem sich stets alle Punkte auf dem gleichen Zoom-Level befinden, wohingegen bei der Clustermaps immer nur einzelne Cluster aufgehen und andere zum Teil gleichzeitig wîeder zugehen wenn man einen neuen Cluster öffnet.

## 5.1) Einfache HeatMap

Auch hier wird wieder dasselbe Vorgehen gewählt, dass eine Funktion für das Erstellen der Karte zur Verfügung gestellt wird, die dann mit beliebigen Teilmengen des Datensatzes aufgerufen werden kann.

In [37]:
def create_Kreis11_HeatMap(df, outputfilename):

    map_11 = folium.Map(location=[Kreis11_lat, Kreis11_lon], zoom_start=13)

    hm_data = [[row.AccidentLocation_WGS84_E, row.AccidentLocation_WGS84_N] for row in df.itertuples()]    

    HeatMap(hm_data,
        name='HeatMap'
    ).add_to(map_11) 

    folium.LayerControl(collapsed=True).add_to(map_11) 
    map_11.save(outputfilename)  

    return map_11

### 5.1.1 Einfache HeatMap für alle Unfälle

In [38]:
create_Kreis11_HeatMap(df_11.copy(), './results/Kreis11_HeatMap_alle_Unfaelle.html')

### 5.1.2 Einfache HeatMap für die Velo-Unfälle

In [39]:
create_Kreis11_HeatMap(df_11[df_11['AccidentInvolvingBicycle'] == True].copy(), 
                           './results/Kreis11_HeatMap_VeloUnfaelle.html')

## 5.2) HeatMap mit zeitlicher Entwicklung

Spannend ist die Möglichkeit, eine HeatMap mit einer zeitlichen Entwicklung zu erzeugen. Hierbei kann der Benutzer durch die einzelnen Zeitpunkte wandern und verfolgen, wie sich die Unfallschwerpunkte im Laufe der Zeit verändern.

### 5.2.1 Entwicklung über die Monate

Als erstes wurde versucht, die Daten im Verlauf des Jahres darzustellen, also pro Monat. Wie immer kann die Auswertung über beliebige Untermenge der Daten vorgenommen werden.

In [40]:
# überprüfen, ob die Monatsangaben noch umsortiert werden müssen
df_11['AccidentMonth_de'].unique()  # werden netterweise grad richtig sortiert ausgegeben...

array(['Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 'Juli',
       'August', 'September', 'Oktober', 'November', 'Dezember'],
      dtype=object)

In [41]:
def create_Kreis11_HeatMapWithTime(df, outputfilename):
    
    map_11 = folium.Map(location=[Kreis11_lat, Kreis11_lon], zoom_start=13)     

    # List comprehension für eine Liste von 12 Listen (eine pro Monat)
    # jede dieser 12 Listen enthält weitere Listen mit den [lat, lon]-Werten der Unfälle im entspr. Monat
    hm_data = [[[row.AccidentLocation_WGS84_E, row.AccidentLocation_WGS84_N] for row in df[df['AccidentMonth'] == month].itertuples()] for month in range(1,13)]

    HeatMapWithTime(
        data=hm_data,
        index=df['AccidentMonth_de'].unique().tolist(),
        auto_play=False,
        name='Heatmap'
    ).add_to(map_11) 


    folium.LayerControl(collapsed=True).add_to(map_11) 

    map_11.save(outputfilename)    
    return map_11


In [42]:
create_Kreis11_HeatMapWithTime(df_11.copy(), './results/Kreis11_HeatMap_alle_Unfaelle_nach_Monaten.html')

In der entstandenen Karte kann nun durch Klicken auf die einfachen Vorwärts- resp. Rückwärtspfeile in der Kontrollleiste unten links der Verlauf der Unfallorte und -häufigkeiten automatisch für alle Monate hintereinander wiedergegeben werden. Wenn man das Tempo der Abfolge selber steuern möchte ist dies mittels Klicken auf die doppelten Pfeile ebenfalls möglich. In der Mitte der Kontrollleiste wird zudem der Name des aktuell dargestellten Monats angezeigt.

Ein Versuch, die zeitliche Entwicklung über die Monate darzustellen erwies sich als nicht besonders spannend, die Verteilung der Unfallorte über die einzelnen Monate erscheint sehr heterogen, es erschliessen sich keine grossen sinnvolle Zusammenhänge. Die Unfälle trugen sich in den verschiedenen Monaten auf (zum Teil erstaunlich disjunkten) Verkehrsachsen zu, was die Vermutung aufkommen lässt, dass dies stark mit aktuell vorhandenen Baustellen oder Strassensperrungen zu tun hat. Dies kann natürlich ebenfalls von Interesse sein, aber nur wenn man zusätzlich die Daten zur Verfügung hat, wann wo Bauarbeiten, Strassensperrungen oder sonstige besondere Vorkommnisse stattgefunden haben.

Im Folgenden wird daher eine HeatMap Serie entwickelt, welche die Daten für die verschiedenen Tageszeit-Kategorien aufzeigt.

### 5.2.2) Entwicklung über die Tageszeit Kategorien

Der ursprüngliche Datensatz der Stadt Zürich wies die Unfallzeit auf eine Stunde genau aus. Beim Einlesen und Vorverarbeiten der Daten (1_Dataset_Load_cleansing.ipynb) wurden diese dann einzelnen Tageszeit-Kategorien zugeordnet. Die unten definierte HeatMap Abfolge zeigt den Verlauf der Unfallorte und -häufigkeiten über diese Tageszeit-Kategorien an.

Korrekterweise muss angefügt werden, dass die Tageszeiten Kategorien nicht alle gleich viele Stunden umfassen und somit die Anzahl Unfälle gewichtet werden müsste mit der umgekehrt proportionalen Länge der entsprechenden Kategorie. Dies ist technisch problemlos möglich, wurde aber der Einfachheit halber für den Moment hier trotzdem weggelassen.

Der Tag ist in folgende Scheiben aufgeteilt:
* Nacht: 0-5h (5 Stunden)
* Rushhour_Morgen: 5-9h (4 Stunden)
* Morgen: 9-12h (3 Stunden)
* Nachmittag: 12-15h (3 Stunden)
* Rushhour_Abend: 15-19h (4 Stunden)
* Abend: 19-24h (5 Stunden)

Dass die Verteilung in die einzelnen Kategorien nicht gleichmässig erfolgt ist, liegt darin begründet, dass sonst einfach in jeder Zeitscheibe "ein bisschen von allem etwas" drin gewesen wäre, also flaue Phasen und Stossverkehr durcheinandergemischt in derselben Zeitscheibe, was eine sinnvolle Auswertung stark erschweren bis verunmöglichen würde. Auch so ist es noch so, dass die kürzer dauernden Kategorien mehr Unfälle aufweisen als die längeren. Würden die Werte noch korrekt gewichtet, wäre dieser Effekt nochmals stärker sichtbar.

In [43]:
# überprüfen, ob die Zeitkategorien noch umsortiert werden müssen
df_11['ZeitKategorie'].unique()  # blöderweise nicht richtig sortiert...

array(['Nachmittag', 'Rushhour_Morgen', 'Rushhour_Abend', 'Abend',
       'Morgen', 'Nacht'], dtype=object)

In [44]:
categories=['Nacht','Rushhour_Morgen','Morgen','Nachmittag','Rushhour_Abend','Abend']

In [45]:
def create_Kreis11_HeatMapWithTime(df, outputfilename):

    map_11 = folium.Map(location=[Kreis11_lat, Kreis11_lon], zoom_start=13)     

    # List comprehension für eine Liste von 6 Listen (eine pro Tageszeit-Kategorie)
    # jede dieser 6 Listen enthält weitere Listen mit den [lat, lon]-Werten der Unfälle in der entspr. Kategorie
    hm_data = [[[row.AccidentLocation_WGS84_E, row.AccidentLocation_WGS84_N] for row in df[df['ZeitKategorie'] == zkat].itertuples()] for zkat in categories]

    HeatMapWithTime(
        data=hm_data,
        index=categories,
        auto_play=False,
        name='Heatmap'
    ).add_to(map_11) 

    folium.LayerControl(collapsed=True).add_to(map_11)  

    map_11.save(outputfilename)  
    return map_11


In [46]:
create_Kreis11_HeatMapWithTime(df_11.copy(), './results/Kreis11_HeatMap_alle_Unfaelle_nach_Tageszeit.html')

Diese Karte zeigt ein deutlich intuitiveres Bild als die Darstellung über die Monate hinweg. Es sind klare Unfallschwerpunkte auszumachen, etwa in der Umgebung des Bahnhofs Oerlikon. Diese bewegen sich von Kategorie zu Kategorie nur wenig, blähen sich aber während der Stosszeiten deutlich aus.

Dieselbe Karte lässt sich natürlich auch nur für die Velounfälle erstellen:

In [47]:
create_Kreis11_HeatMapWithTime(df_11[df_11['AccidentInvolvingBicycle'] == True].copy(),
                               './results/Kreis11_HeatMap_Velo-Unfaelle_nach_Tageszeit.html')

Hier sieht man wieder etwas mehr Bewegung als bei der Sicht auf alle Unfälle. Einerseits hat dies sicher damit zu tun, dass es  weniger Velo-Unfälle gibt als Unfälle überhaupt und somit hier noch mehr statistische Schwankungen zu erwarten sind. Anderseits ist es aber auch gut möglich, dass Velo-Aktivitäten je nach Tageszeit an unterschiedlichen Orten stattfinden. So bewegen sich die Velofahrer in Affoltern offenbar am Vormittag deutlich weniger (oder aber zumindest vorsichtiger) als am Nachmittag. Diese Tatsache liesse sich unter Umständen nutzen, indem zum Beispiel überprüft werden könnte, ob die Verkehrsströme je nach Tageszeit etwas anders gelenkt werden könnten um kritische Gebiete zu entlasten.